<a href="https://colab.research.google.com/github/TejasP/data/blob/master/OptionAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from nsepy import get_history
yr=2018
mn=1
start_date = date(2019,1,1)
end_date = date(2019,3,31)
from nsepy.derivatives import get_expiry_date
expiry = get_expiry_date(year=yr, month=mn)
expiry = sorted(expiry)

In [ ]:
!pip install nsepy

  Created wheel for nsepy: filename=nsepy-0.8-cp36-none-any.whl size=36084 sha256=50dfdff6cf71491e1d9e67a09a786d40396a5befce6dc84db66834eee1f9bc2a
  Stored in directory: /root/.cache/pip/wheels/05/2c/72/5324e267426eea65dec0c21a12a48b15887c0cf686c7a8fe11
Successfully built nsepy


In [ ]:
nifty = get_history(symbol="NIFTY",start=start_date,end=end_date,index=True)

KeyboardInterrupt: ignored

In [ ]:
nifty

In [ ]:
nifty["Average"] = nifty[["Open","High","Low","Close"]].mean(axis=1)
nifty["Date"] = pd.to_datetime(nifty.index)
nifty
def round_number(num,divisor):
    if num%divisor< 25:
        return num-(num%divisor)
    if num%divisor > 25:
        return num+(divisor-(num%divisor))

def getExpiry(row):
    yr = row.Date.year
    mn = row.Date.month
    expiryDay = sorted(get_expiry_date(year=yr, month=mn))[-1]
    
    if expiryDay.day < row.Date.day:
        if row.Date.month > 12 :
            yr = 1 
            mn =  row.Date.year+1
        else:
            mn = row.Date.month+1 
        expiryDay = sorted(get_expiry_date(year=yr, month=mn))[-1]
    return expiryDay

def getOptionData(row):
    print("Getting OptionData")
    nifty_opt = get_history(symbol="NIFTY",start=row.Date,end=row.Date,index=True,option_type='CE',strike_price=round_number(row.Average,50),expiry_date=row.Expiry)
    if (len(nifty_opt.index) == 0):
        ACO = 0
        ACC = 0
    else :
        ACO = nifty_opt.iloc[0,4]
        ACC = nifty_opt.iloc[0,7]
    
    nifty_opt = get_history(symbol="NIFTY",start=row.Date,end=row.Date,index=True,option_type='PE',strike_price=round_number(row.Average,50),expiry_date=row.Expiry)
    if (len(nifty_opt.index) == 0):
        APO = 0
        APC = 0
    else :
        APO = nifty_opt.iloc[0,4]
        APC = nifty_opt.iloc[0,7]
    return pd.Series([ACO,ACC,APO,APC])


def getOptionData1SD(row):
    print("Getting OptionData1SD")
    ATM = round_number(row.Average,50)
    CE_STRIKE = ATM + 150
    PE_STRIKE = ATM - 150
    
    nifty_opt = get_history(symbol="NIFTY",start=row.Date,end=row.Date,index=True,option_type='CE',strike_price=CE_STRIKE,expiry_date=row.Expiry)
    if (len(nifty_opt.index) == 0):
        ACO = 0
        ACC = 0
    else :
        ACO = nifty_opt.iloc[0,4]
        ACC = nifty_opt.iloc[0,7]
    
    nifty_opt = get_history(symbol="NIFTY",start=row.Date,end=row.Date,index=True,option_type='PE',strike_price=PE_STRIKE,expiry_date=row.Expiry)
    if (len(nifty_opt.index) == 0):
        APO = 0
        APC = 0
    else :
        APO = nifty_opt.iloc[0,4]
        APC = nifty_opt.iloc[0,7]
    return pd.Series([ACO,ACC,APO,APC])


nifty["Expiry"] = nifty.apply(lambda x: getExpiry(x), axis=1)
nifty[["ATM_CE_OPEN", "ATM_CE_CLOSE","ATM_PE_OPEN","ATM_PE_CLOSE"]] = nifty.apply(lambda x: getOptionData(x),axis=1)
nifty[["1SD_CE_OPEN", "1SD_CE_CLOSE","1SD_PE_OPEN","1SD_PE_CLOSE"]] = nifty.apply(lambda x: getOptionData1SD(x),axis=1)

nifty

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

ax = plt.gca()
nifty['ATM_OPEN_SUM'] = nifty['ATM_CE_OPEN']+ nifty['ATM_PE_OPEN']
nifty['ATM_CLOSE_SUM'] = nifty['ATM_CE_CLOSE']+ nifty['ATM_PE_CLOSE']

nifty['1SD_OPEN_SUM'] = nifty['1SD_CE_OPEN']+ nifty['1SD_PE_OPEN']
nifty['1SD_CLOSE_SUM'] = nifty['1SD_CE_CLOSE']+ nifty['1SD_PE_CLOSE']

nifty['returns'] = round((nifty['ATM_OPEN_SUM'] -nifty['ATM_CLOSE_SUM']),2)
nifty['creturns'] = nifty['returns'].cumsum()

nifty.plot(kind='line',x='Date',y=['creturns'],ax=ax,figsize=(15,15))

#nifty.plot(kind='bar',x='Date',y=['ATM_OPEN_SUM'],ax=ax,figsize=(15,15))
#nifty.plot(kind='bar',x='Date',y=['ATM_CLOSE_SUM'], color='red', ax=ax,figsize=(15,15))

plt.show()

In [ ]:
nifty

In [ ]:
import pyfolio as pf
nifty.returns = nifty.returns.round()
pf.create_simple_tear_sheet(nifty.returns)

In [ ]:
nifty.to_csv('nifty.csv', index=False)

nifty.creturns